In [1]:
import pandas as pd
import numpy as np

In [3]:
datafn = pd.read_csv("full_variants/caqtls.african.lcls.benchmarking.all.tsv", sep="\t", header=0)


In [5]:
#np.log10(9.94985e-06)
#9.94985e-06

In [6]:
datafn["log10p"] = np.log10(datafn["obs.pval"])*-1
threshold=5

dataf1 = datafn[datafn["log10p"]>threshold]
dataf1["pos0"] = dataf1["var.pos_hg38"] - 1
peaks_var = dataf1[["var.chr", "pos0", "var.pos_hg38"]].reset_index(drop=True)
#dataf1



/users/anusri/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [7]:
import pybedtools
peaks_data_merged=pd.read_csv("/mnt/lab_data2/anusri/chromatin-atlas-anvil/data/afgr_data/peak_files/merged.peaks.af.top.50k.bed", sep="\t", header=None)
#print(peaks_data_merged.head())
beds_var = pybedtools.BedTool.from_dataframe(peaks_var)

dataf1 = pybedtools.BedTool.from_dataframe(dataf1)
peak_bed = pybedtools.BedTool.from_dataframe(peaks_data_merged) # this is a merged peaks file
peaks_invar = peak_bed.intersect(beds_var, c=True)

In [8]:
peaks_invar = peaks_invar.to_dataframe()

In [9]:
peaks_invar = peaks_invar[peaks_invar["name"]>0]

In [10]:
datafn["pos0"] = datafn["var.pos_hg38"] - 1
all_var = datafn[["var.chr", "pos0", "var.pos_hg38"]].reset_index(drop=True)
beds_var = pybedtools.BedTool.from_dataframe(all_var)
peaks_invar = pybedtools.BedTool.from_dataframe(peaks_invar)
var_inpeaks = beds_var.intersect(peaks_invar, c=True)

In [11]:
var_inpeaks = var_inpeaks.to_dataframe()

In [12]:
var_inpeaks.shape

(219382, 4)

In [13]:
datafn["sigpeaks"] = var_inpeaks["name"]>0

In [14]:
datafn.head(1)

,var.chr,var.pos_hg38,var.allele1,var.allele2,var.isused,obs.pval,obs.beta,obs.label,pred.enformer.encsr000emt.variantscore.global_sad,pred.enformer.encsr000emt.variantscore.global_sar,...,var.snp_id,var.region,var.snp,var.dbsnp_rsid,obs.distance,obs.se,obs.abs_beta,log10p,pos0,sigpeaks
0,chr1,866281,C,T,False,0.97428,0.006546,False,0.000051,0.000068,...,1_866281_C_T,1:866099:866351,1_866281_C_T,rs12132974,181,0.202449,0.006546,0.011316,866280,False


In [15]:
datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])].shape

(3440, 42)

In [16]:
sum(datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])]["var.isused"])

2827

In [17]:
datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])]["var.isused"]

59         True
64         True
65        False
68         True
125        True
          ...  
218887     True
218888     True
218889     True
219028     True
219132     True
Name: var.isused, Length: 3440, dtype: bool

In [18]:
datafn.loc[(datafn["log10p"]<3) & (datafn["sigpeaks"]),"var.isused"] = False

In [19]:
datafn[(datafn["log10p"]<3) & (datafn["sigpeaks"])]["var.isused"]

59        False
64        False
65        False
68        False
125       False
          ...  
218887    False
218888    False
218889    False
219028    False
219132    False
Name: var.isused, Length: 3440, dtype: bool

In [20]:
datafn.head(1)

,var.chr,var.pos_hg38,var.allele1,var.allele2,var.isused,obs.pval,obs.beta,obs.label,pred.enformer.encsr000emt.variantscore.global_sad,pred.enformer.encsr000emt.variantscore.global_sar,...,var.snp_id,var.region,var.snp,var.dbsnp_rsid,obs.distance,obs.se,obs.abs_beta,log10p,pos0,sigpeaks
0,chr1,866281,C,T,False,0.97428,0.006546,False,0.000051,0.000068,...,1_866281_C_T,1:866099:866351,1_866281_C_T,rs12132974,181,0.202449,0.006546,0.011316,866280,False


In [21]:
datafn.shape

(219382, 42)

In [22]:
datafn.to_csv("full_variants/caqtls.african.lcls.benchmarking.all.addfilter.tsv", sep="\t", header=True, index=False)